### Importing the essential libraries...

In [109]:
import numpy as np
import re
import pandas as pd
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [110]:
news_df_F = pd.read_csv('fake.csv')
news_df_T = pd.read_csv('True.csv')

In [111]:
news_df_F.head()

,title,text,subject,date
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017"
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017"
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017"
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017"
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017"


In [112]:
news_df_T.head()

,title,text,subject,date
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017"
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017"
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017"
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017"
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017"


### Exploratory Data Analysis...

In [116]:
news_df_F.isnull().sum()

title      0
text       0
subject    0
date       0
dtype: int64

In [117]:
news_df_T.isnull().sum()

title      0
text       0
subject    0
date       0
dtype: int64

In [118]:
news_df_F.shape

(23481, 4)

In [119]:
news_df_T.shape

(21417, 4)

In [120]:
news_df_F = news_df_F.fillna(' ')

In [121]:
news_df_T = news_df_T.fillna(' ')

In [122]:
news_df_F.isnull().sum()

title      0
text       0
subject    0
date       0
dtype: int64

In [123]:
news_df_T.isnull().sum()

title      0
text       0
subject    0
date       0
dtype: int64

In [124]:
news_df_F['content'] = news_df_F['title']+' '+news_df_F['title']

In [125]:
news_df_F['label'] = 1

In [126]:
news_df_F

,title,text,subject,date,content,label
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017",Donald Trump Sends Out Embarrassing New Year’...,1
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017",Drunk Bragging Trump Staffer Started Russian ...,1
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017",Sheriff David Clarke Becomes An Internet Joke...,1
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017",Trump Is So Obsessed He Even Has Obama’s Name...,1
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017",Pope Francis Just Called Out Donald Trump Dur...,1
...,...,...,...,...,...,...
23476,McPain: John McCain Furious That Iran Treated ...,21st Century Wire says As 21WIRE reported earl...,Middle-east,"January 16, 2016",McPain: John McCain Furious That Iran Treated ...,1
23477,JUSTICE? Yahoo Settles E-mail Privacy Class-ac...,21st Century Wire says It s a familiar theme. ...,Middle-east,"January 16, 2016",JUSTICE? Yahoo Settles E-mail Privacy Class-ac...,1
23478,Sunnistan: US and Allied ‘Safe Zone’ Plan to T...,Patrick Henningsen 21st Century WireRemember ...,Middle-east,"January 15, 2016",Sunnistan: US and Allied ‘Safe Zone’ Plan to T...,1
23479,How to Blow $700 Million: Al Jazeera America F...,21st Century Wire says Al Jazeera America will...,Middle-east,"January 14, 2016",How to Blow $700 Million: Al Jazeera America F...,1


In [127]:
news_df_T['content'] = news_df_T['title']+' '+news_df_T['title']

In [128]:
news_df_T['label'] = 0

In [129]:
news_df_T

,title,text,subject,date,content,label
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017","As U.S. budget fight looms, Republicans flip t...",0
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017",U.S. military to accept transgender recruits o...,0
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017",Senior U.S. Republican senator: 'Let Mr. Muell...,0
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017",FBI Russia probe helped by Australian diplomat...,0
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017",Trump wants Postal Service to charge 'much mor...,0
...,...,...,...,...,...,...
21412,'Fully committed' NATO backs new U.S. approach...,BRUSSELS (Reuters) - NATO allies on Tuesday we...,worldnews,"August 22, 2017",'Fully committed' NATO backs new U.S. approach...,0
21413,LexisNexis withdrew two products from Chinese ...,"LONDON (Reuters) - LexisNexis, a provider of l...",worldnews,"August 22, 2017",LexisNexis withdrew two products from Chinese ...,0
21414,Minsk cultural hub becomes haven from authorities,MINSK (Reuters) - In the shadow of disused Sov...,worldnews,"August 22, 2017",Minsk cultural hub becomes haven from authorit...,0
21415,Vatican upbeat on possibility of Pope Francis ...,MOSCOW (Reuters) - Vatican Secretary of State ...,worldnews,"August 22, 2017",Vatican upbeat on possibility of Pope Francis ...,0


In [130]:
news_df = news_df_F + news_df_T

In [131]:
news_df.head()

,title,text,subject,date,content,label
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,NewspoliticsNews,"December 31, 2017December 31, 2017",Donald Trump Sends Out Embarrassing New Year’...,1.0
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,NewspoliticsNews,"December 31, 2017December 29, 2017",Drunk Bragging Trump Staffer Started Russian ...,1.0
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",NewspoliticsNews,"December 30, 2017December 31, 2017",Sheriff David Clarke Becomes An Internet Joke...,1.0
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",NewspoliticsNews,"December 29, 2017December 30, 2017",Trump Is So Obsessed He Even Has Obama’s Name...,1.0
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,NewspoliticsNews,"December 25, 2017December 29, 2017",Pope Francis Just Called Out Donald Trump Dur...,1.0


In [132]:
X = news_df.drop('label',axis=1)
y = news_df['label']

In [133]:
X = X[:1]

In [134]:
X.shape

(1, 5)

### Text Preprocessing...

In [135]:
ps = PorterStemmer()
def stemming(content):
    stemmed_content = re.sub('[^a-zA-Z]',' ',content)
    stemmed_content = stemmed_content.lower()
    stemmed_content = stemmed_content.split()
    stemmed_content = [ps.stem(word) for word in stemmed_content if not word in stopwords.words('english')]
    stemmed_content = ' '.join(stemmed_content)
    return stemmed_content

In [ ]:
news_df['content'] = news_df['content'].apply(stemming)

In [ ]:
news_df['content']

In [ ]:
X = news_df['content'].values
y = news_df['label'].values

In [ ]:
vector = TfidfVectorizer()
vector.fit(X)
X = vector.transform(X)

In [ ]:
print(X)

### Train_Test Split...

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X, y, test_size = 0.2, stratify=y, random_state=2)

In [ ]:
X_train.shape

### Model Training...

In [ ]:
model = LogisticRegression()
model.fit(X_train,Y_train)

In [ ]:
train_y_pred = model.predict(X_train)
print(accuracy_score(train_y_pred,Y_train))

In [ ]:
testing_y_pred = model.predict(X_test)
print(accuracy_score(testing_y_pred,Y_test))

In [ ]:
input_data = X_test[10]
prediction = model.predict(input_data)

### Prediction...

In [ ]:
if prediction[0] == 0:
    print('The News Is Real')
else:
    print('The News is Fake')

In [ ]:
news_df['content'][2]